In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
import numpy
import pandas

from IPython.display import display, clear_output

from tqdm.notebook import tqdm

import copy

import swasputils

In [3]:
harmonic_fractions = [ 1/x for x in range(1, 100) ]
beat_offsets = [ 86400 * n for n in range(1, 100) ]

In [4]:
folded_lightcurves = swasputils.FoldedLightcurves()

In [5]:
source_groups = folded_lightcurves.df.groupby('SWASP ID')

In [6]:
grouped_ids = []
grouped_periods = []
grouped_harmonics = []
grouped_harmonic_types = []

for swasp_id, df in tqdm(source_groups):
    periods = df['Period']
    harmonic_group = 1
    while True:
        if len(periods) == 0:
            break
        max_period = max(periods)
        period_ratios = [ p/max_period for p in periods ]

        is_harmonic = []
        for period, ratio in zip(periods, period_ratios):
            result = False
            for frac in harmonic_fractions:
                if numpy.isclose(ratio, frac, rtol=1e-1): #To do: Figure out a justifiable value for this! especially wrt harmonic_fractions
                    result = True
                    if frac == 1.0:
                        grouped_harmonic_types.append('fundamental')
                    else:
                        grouped_harmonic_types.append('harmonic')
                    break
            if not result:
                for beat in beat_offsets:
                    if numpy.isclose(period + beat, max_period, rtol=1e-1):
                        result = True
                        grouped_harmonic_types.append('day beat')
                        break
            if result:
                grouped_ids.append(swasp_id)
                grouped_periods.append(period)
                grouped_harmonics.append(harmonic_group)
            is_harmonic.append(result)
            
        periods = [ p for p, h in zip(periods, is_harmonic) if not h ]
        harmonic_group += 1

  0%|          | 0/770735 [00:00<?, ?it/s]

In [7]:
harmonics_table = folded_lightcurves.df.merge(pandas.DataFrame({
    'SWASP ID': grouped_ids,
    'Period': grouped_periods,
    'Harmonic Group': grouped_harmonics,
    'Harmonic Type': grouped_harmonic_types,
}))
harmonics_table

,Period Number,Period,Sigma,Chi Squared,SWASP ID,Harmonic Group,Harmonic Type
0,1,2.480236e+04,3.82,58.50,1SWASPJ000000.15+320847.6,1,fundamental
1,1,1.884039e+06,4.07,47.49,1SWASPJ000000.74+561358.9,1,fundamental
2,3,4.280812e+05,4.22,31.22,1SWASPJ000000.74+561358.9,1,harmonic
3,5,2.758600e+05,4.19,38.11,1SWASPJ000000.74+561358.9,1,harmonic
4,2,3.232903e+05,3.78,29.66,1SWASPJ000001.10+562043.7,1,fundamental
...,...,...,...,...,...,...,...
1600997,1,1.383512e+07,4.06,135.83,1SWASPJ215657.91-310826.5,1,fundamental
1600998,2,1.247697e+07,4.13,55.57,1SWASPJ215657.91-310826.5,1,fundamental
1600999,3,9.040948e+06,4.06,80.41,1SWASPJ215657.91-310826.5,1,day beat
1601000,1,9.036595e+06,4.91,169.59,1SWASPJ215658.98-260752.2,1,fundamental


In [8]:
harmonics_table.to_csv(os.path.join(swasputils.CACHE_LOCATION, 'harmonic_groups.csv'))

In [9]:
harmonics_table.to_pickle(os.path.join(swasputils.CACHE_LOCATION, 'harmonic_groups.pickle'))